In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q imbalanced-learn scikit-learn

# ハイブリッド量子強化アンサンブル分類（電力グリッド安定性ワークフロー）

*使用量の目安：Eagle r3プロセッサ上の各ジョブにつきQPU時間で約20分。（注意：これはあくまで目安です。実際の実行時間は異なる場合があります。）*
## 背景
このチュートリアルでは、古典アンサンブルを量子最適化ステップで強化するハイブリッド量子・古典ワークフローを実演します。Multiverse Computingの「Singularity Machine Learning – Classification」（Qiskit Function）を使用して、従来型の学習器プール（例：決定木、k-NN、ロジスティック回帰）を訓練し、その後量子レイヤーでプールを精錬することで多様性と汎化性能を向上させます。目的は実用的なものです。実際のグリッド安定性予測タスクにおいて、同じデータ分割の下で強力な古典ベースラインと量子最適化された代替手法を比較し、量子ステップがどこで有効か、またそのコストがどの程度かを確認できるようにします。

これが重要である理由は以下の通りです。多数の弱学習器から優れたサブセットを選択することは、アンサンブルサイズに応じて急速に増大する組合せ最適化問題です。ブースティング、バギング、スタッキングなどの古典的ヒューリスティクスは中規模では良好に機能しますが、大規模で冗長なモデルライブラリを効率的に探索するのは困難になることがあります。この関数は量子アルゴリズム、具体的にはQAOA（および他の構成ではオプションでVQE）を統合し、古典学習器の訓練後にその探索空間をより効果的に探索することで、汎化性能に優れたコンパクトで多様なサブセットを見つける可能性を高めます。

重要な点として、データスケールは量子ビット数による制限を受けません。データに関する重い処理（前処理、学習器プールの訓練、評価）は古典的なまま行われ、数百万のサンプルを扱うことができます。量子ビットは量子選択ステップで使用されるアンサンブルサイズのみを決定します。この分離こそが、現在のハードウェアでこのアプローチを実用的にしている理由です。データとモデル訓練には馴染みのあるscikit-learnワークフローを維持しながら、Qiskit Functionsのクリーンなアクションインターフェースを通じて量子ステップを呼び出すことができます。

実際には、アンサンブルに様々な種類の学習器を提供できます（例：決定木、ロジスティック回帰、k-NNなど）が、決定木が最も良い性能を発揮する傾向があります。オプティマイザは一貫してより強力なアンサンブルメンバーを優先し、異種の学習器が提供された場合、線形回帰器のような弱いモデルは通常、決定木のようなより表現力の高いモデルに置き換えられて除去されます。

このチュートリアルで行うこと：グリッド安定性データセットの準備とバランス調整、古典的なAdaBoostベースラインの確立、アンサンブル幅と正則化を変化させた複数の量子構成の実行、Qiskit Serverless経由でのIBM&reg;シミュレータまたはQPU上での実行、そしてすべての実行における精度、適合率、再現率、F1スコアの比較を行います。この過程で、関数のアクションパターン（`create`、`fit`、`predict`、`fit_predict`、`create_fit_predict`）と主要な制御パラメータを使用します：
- 正則化タイプ：直接的なスパース化のための`onsite`（λ）と、相互作用項とオンサイト項の比率ベースのトレードオフのための`alpha`
- 自動正則化：目標選択比率を指定して`regularization="auto"`を設定し、スパース性を自動的に適応させます
- オプティマイザオプション：シミュレータ対QPU、反復回数、古典オプティマイザとそのオプション、トランスパイレーション深度、ランタイムサンプラー/エスティメータ設定

ドキュメントのベンチマークによると、困難な問題において学習器数（量子ビット数）が増加するにつれて精度が向上し、量子分類器は同等の古典アンサンブルと同等以上の性能を達成しています。このチュートリアルでは、ワークフロー全体をエンドツーエンドで再現し、アンサンブル幅の増加や適応的正則化への切り替えが、合理的なリソース使用量でより良いF1スコアをもたらすタイミングを検証します。その結果として、量子最適化ステップが実際のアプリケーションにおいて古典的アンサンブル学習を置き換えるのではなく、いかに補完できるかについての実践的な理解が得られます。
## 要件
このチュートリアルを開始する前に、Python環境に以下のパッケージがインストールされていることを確認してください：

- `qiskit[visualization]~=2.1.0`
- `qiskit-serverless~=0.24.0`
- `qiskit-ibm-runtime v0.40.1`
- `qiskit-ibm-catalog~=0.8.0`
- `scikit-learn==1.5.2`
- `pandas>=2.0.0,<3.0.0`
- `imbalanced-learn~=0.12.3`
## セットアップ
このセクションでは、Qiskit Serverlessクライアントを初期化し、Multiverse Computingが提供するSingularity Machine Learning – Classification関数をロードします。
Qiskit Serverlessを使用すると、リソース管理を気にすることなく、IBMのマネージドクラウドインフラストラクチャ上でハイブリッド量子・古典ワークフローを実行できます。
認証してQiskit Functionsにアクセスするには、IBM Quantum PlatformのAPIキーとクラウドリソース名（CRN）が必要です。
### データセットのダウンロード
このチュートリアルを実行するために、ラベル付きの電力系統センサーデータを含む前処理済みの**グリッド安定性分類データセット**を使用します。
以下のセルは、必要なフォルダ構造を自動的に作成し、`wget`を使用してトレーニングファイルとテストファイルの両方を環境に直接ダウンロードします。
これらのファイルが既にローカルにある場合でも、バージョンの一貫性を確保するために安全に上書きされます。

In [1]:
## Download dataset for Grid Stability Classification

# Create data directory if it doesn't exist
!mkdir -p data_tutorial/grid_stability

# Download the training and test sets from the official Qiskit documentation repo
!wget -q --show-progress -O data_tutorial/grid_stability/train.csv \
  https://raw.githubusercontent.com/Qiskit/documentation/main/datasets/tutorials/grid_stability/train.csv

!wget -q --show-progress -O data_tutorial/grid_stability/test.csv \
  https://raw.githubusercontent.com/Qiskit/documentation/main/datasets/tutorials/grid_stability/test.csv

# Check the files have been downloaded
!echo "Dataset files downloaded:"
!ls -lh data_tutorial/grid_stability/*.csv

data_tutorial/grid_ 100%[===================>] 612.94K  --.-KB/s    in 0.01s   
data_tutorial/grid_ 100%[===================>] 108.19K  --.-KB/s    in 0.006s  
Dataset files downloaded:
-rw-r--r-- 1 coder coder 109K Nov  8 18:50 data_tutorial/grid_stability/test.csv
-rw-r--r-- 1 coder coder 613K Nov  8 18:50 data_tutorial/grid_stability/train.csv


### Import required packages

In this section, we import all Python packages and Qiskit modules used throughout the tutorial.
These include core scientific libraries for data handling and model evaluation - such as `NumPy`, `pandas`, and `scikit-learn` - along with visualization tools and Qiskit components for running the quantum-enhanced model.
We also import the `QiskitRuntimeService` and `QiskitFunctionsCatalog` to connect with IBM Quantum&reg; services and access the Singularity Machine Learning function.

In [2]:
from typing import Tuple
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from qiskit_ibm_catalog import QiskitFunctionsCatalog
from qiskit_ibm_runtime import QiskitRuntimeService
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

### 必要なパッケージのインポート
このセクションでは、チュートリアル全体で使用するすべてのPythonパッケージとQiskitモジュールをインポートします。
これらには、データ処理とモデル評価のための主要な科学計算ライブラリ（`NumPy`、`pandas`、`scikit-learn`など）、可視化ツール、および量子強化モデルを実行するためのQiskitコンポーネントが含まれます。
また、IBM Quantum&reg;サービスに接続しSingularity Machine Learning関数にアクセスするために、`QiskitRuntimeService`と`QiskitFunctionsCatalog`もインポートします。

In [3]:
IBM_TOKEN = ""
IBM_INSTANCE_TEST = ""
IBM_INSTANCE_QUANTUM = ""
FUNCTION_NAME = "multiverse/singularity"
RANDOM_STATE: int = 123
TRAIN_PATH = "data_tutorial/grid_stability/train.csv"
TEST_PATH = "data_tutorial/grid_stability/test.csv"

### 定数変数の設定

In [4]:
service = QiskitRuntimeService(
    token=IBM_TOKEN,
    channel="ibm_quantum_platform",
    instance=IBM_INSTANCE_QUANTUM,
)

backend = service.least_busy()
catalog = QiskitFunctionsCatalog(
    token=IBM_TOKEN,
    instance=IBM_INSTANCE_TEST,
    channel="ibm_quantum_platform",
)
singularity = catalog.load(FUNCTION_NAME)
print(
    "Successfully connected to IBM Qiskit Serverless and loaded the Singularity function."
)
print("Catalog:", catalog)
print("Singularity function:", singularity)

Successfully connected to IBM Qiskit Serverless and loaded the Singularity function.
Catalog: <QiskitFunctionsCatalog>
Singularity function: QiskitFunction(multiverse/singularity)


### IBM Quantumへの接続とSingularity関数のロード
次に、IBM Quantumサービスで認証を行い、Qiskit Functions CatalogからSingularity Machine Learning – Classification関数をロードします。
`QiskitRuntimeService`は、APIトークンとインスタンスCRNを使用してIBM Quantum Platformへの安全な接続を確立し、量子バックエンドへのアクセスを可能にします。
その後、`QiskitFunctionsCatalog`を使用して名前（`"multiverse/singularity"`）でSingularity関数を取得し、後でハイブリッド量子・古典計算に呼び出せるようにします。
セットアップが成功すると、関数が正しくロードされたことを示す確認メッセージが表示されます。

In [5]:
def load_data(data_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Load data from the given path to X and y arrays."""
    df: pd.DataFrame = pd.read_csv(data_path)
    return df.iloc[:, :-1].values, df.iloc[:, -1].values


def evaluate_predictions(predictions, y_true):
    """Compute and print accuracy, precision, recall, and F1 score."""
    accuracy = accuracy_score(y_true, predictions)
    precision = precision_score(y_true, predictions)
    recall = recall_score(y_true, predictions)
    f1 = f1_score(y_true, predictions)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1)
    return accuracy, precision, recall, f1

## Step 1: Map classical inputs to a quantum problem

We begin by preparing the dataset for hybrid quantum–classical experimentation. The goal of this step is to convert the raw grid-stability data into balanced training, validation, and test splits that can be used consistently by both classical and quantum workflows. Maintaining identical splits ensures that later performance comparisons are fair and reproducible.

### Data loading and preprocessing

We first load the training and test CSV files, create a validation split, and balance the dataset using random over-sampling. Balancing prevents bias toward the majority class and provides a more stable learning signal for both classical and quantum ensemble models.

In [6]:
# Load and upload the data
X_train, y_train = load_data(TRAIN_PATH)
X_test, y_test = load_data(TEST_PATH)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE
)

# Balance the dataset through over-sampling of the positive class
ros = RandomOverSampler(random_state=RANDOM_STATE)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print("Shapes:")
print("  X_train_bal:", X_train_bal.shape)
print("  y_train_bal:", y_train_bal.shape)
print("  X_val:", X_val.shape)
print("  y_val:", y_val.shape)
print("  X_test:", X_test.shape)
print("  y_test:", y_test.shape)

Shapes:
  X_train_bal: (5104, 12)
  y_train_bal: (5104,)
  X_val: (850, 12)
  y_val: (850,)
  X_test: (750, 12)
  y_test: (750,)


### ヘルパー関数の定義
メインの実験を実行する前に、データの読み込みとモデル評価を効率化するいくつかの小さなユーティリティ関数を定義します。
- `load_data()`は、入力CSVファイルをNumPy配列に読み込み、`scikit-learn`および量子ワークフローとの互換性のために特徴量とラベルを分離します。
- `evaluate_predictions()`は、主要なパフォーマンス指標（精度、適合率、再現率、F1スコア）を計算し、タイミング情報が提供された場合はオプションで実行時間を報告します。

これらのヘルパー関数は、ノートブックの後半で繰り返される操作を簡略化し、古典分類器と量子分類器の両方で一貫した指標の報告を保証します。

In [7]:
# ----- Classical baseline: AdaBoost -----
baseline = AdaBoostClassifier(n_estimators=60, random_state=RANDOM_STATE)
baseline.fit(X_train_bal, y_train_bal)
baseline_pred = baseline.predict(X_test)
print("Classical AdaBoost baseline:")
_ = evaluate_predictions(baseline_pred, y_test)

Classical AdaBoost baseline:
Accuracy: 0.7893333333333333
Precision: 1.0
Recall: 0.7893333333333333
F1: 0.8822652757078987


## ステップ1：古典的な入力を量子問題にマッピングする
ハイブリッド量子・古典実験のためにデータセットを準備するところから始めます。このステップの目的は、生のグリッド安定性データを、古典ワークフローと量子ワークフローの両方で一貫して使用できるバランスの取れた訓練セット、検証セット、テストセットに変換することです。同一の分割を維持することで、後のパフォーマンス比較が公平かつ再現可能になります。
### データの読み込みと前処理
まず、トレーニングとテストのCSVファイルを読み込み、検証用分割を作成し、ランダムオーバーサンプリングを使用してデータセットのバランスを調整します。バランス調整により多数派クラスへの偏りを防ぎ、古典および量子アンサンブルモデルの両方により安定した学習信号を提供します。

In [8]:
# QAOA / runtime configuration for best results on hardware
optimizer_options = {
    "simulator": False,  # set True to test locally without QPU
    "num_solutions": 100_000,  # broaden search over candidate ensembles
    "reps": 3,  # QAOA depth (circuit layers)
    "optimization_level": 3,  # transpilation effort
    "num_transpiler_runs": 30,  # explore multiple layouts
    "classical_optimizer": "COBYLA",  # robust default for this landscape
    "classical_optimizer_options": {
        "maxiter": 60  # practical convergence budget
    },
    # You can pass backend-specific options; leaving None uses least-busy routing
    "estimator_options": None,
    "sampler_options": None,
}

print("Configured hardware optimization profile:")
for key, value in optimizer_options.items():
    print(f"  {key}: {value}")

Configured hardware optimization profile:
  simulator: False
  num_solutions: 100000
  reps: 3
  optimization_level: 3
  num_transpiler_runs: 30
  classical_optimizer: COBYLA
  classical_optimizer_options: {'maxiter': 60}
  estimator_options: None
  sampler_options: None


## Step 3: Execute using Qiskit primitives

We now execute the full workflow using the Singularity function’s `create_fit_predict` action to train, optimize, and evaluate the `QuantumEnhancedEnsembleClassifier` end-to-end on IBM infrastructure. The function builds the ensemble, applies quantum optimization through Qiskit primitives, and returns both predictions and job metadata (including runtime and resource usage). The classical data split from Step 1 is reused for reproducibility, with validation data passed through `fit_params` so the optimization can tune hyperparameters internally while keeping the held-out test set untouched.

In this step, we explore several configurations of the quantum ensemble to understand how key parameters - specifically `num_learners` and `regularization` - affect both result quality and QPU usage.
- `num_learners` determines the ensemble width (and implicitly, the number of qubits), influencing the model’s capacity and computational cost.
- `regularization` controls sparsity and overfitting, shaping how many learners remain active after optimization.

By varying these parameters, we can see how ensemble width and regularization interact: increasing width typically improves F1 but costs more QPU time, while stronger or adaptive regularization can improve generalization at roughly the same hardware footprint. The next subsections walk through three representative configurations to illustrate these effects.

### Baseline

This configuration uses `num_learners = 10` and `regularization = 7`.

- `num_learners` controls the ensemble width — effectively the number of weak learners combined and, on quantum hardware, the **number of qubits required**. A larger value expands the combinatorial search space and can improve accuracy and recall, but also increases circuit width, compilation time, and overall QPU usage.
- `regularization` sets the penalty strength for including additional learners. With the default "onsite" regularization, higher values enforce stronger sparsity (fewer learners kept), while lower values allow more complex ensembles.

This setup provides a low-cost baseline, showing how a small ensemble behaves before scaling width or tuning sparsity.

In [9]:
# Problem scale and regularization
NUM_LEARNERS = 10
REGULARIZATION = 7

In [10]:
# ----- Quantum-enhanced ensemble on IBM hardware -----
print("\n-- Submitting quantum-enhanced ensemble job --")
job_1 = singularity.run(
    action="create_fit_predict",
    name="grid_stability_qeec",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=NUM_LEARNERS,
    regularization=REGULARIZATION,
    optimizer_options=optimizer_options,  # from Step 2
    backend_name=backend,  # least-busy compatible backend
    instance=IBM_INSTANCE_QUANTUM,
    random_state=RANDOM_STATE,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_test=X_test,
    fit_params={"validation_data": (X_val, y_val)},
    options={"save": False},
)
result_1 = job_1.result()
print("Action status:", result_1.get("status"))
print("Action message:", result_1.get("message"))
print("Metadata:", result_1.get("metadata"))
qeec_pred_job_1 = np.array(result_1["data"]["predictions"])
_ = evaluate_predictions(qeec_pred_job_1, y_test)


-- Submitting quantum-enhanced ensemble job --
Action status: ok
Action message: Classifier created, fitted, and predicted.
Metadata: {'resource_usage': {'RUNNING: MAPPING': {'CPU_TIME': 267.05158376693726}, 'RUNNING: WAITING_QPU': {'CPU_TIME': 3336.8785166740417}, 'RUNNING: POST_PROCESSING': {'CPU_TIME': 152.4274561405182}, 'RUNNING: EXECUTING_QPU': {'QPU_TIME': 1550.1889700889587}}}
Accuracy: 0.868
Precision: 1.0
Recall: 0.868
F1: 0.9293361884368309


In [11]:
status_1 = job_1.status()
print("\nQuantum job status:", status_1)


Quantum job status: DONE


## ステップ2：量子ハードウェア実行のための問題の最適化
アンサンブル選択タスクは組合せ最適化問題として定式化されます。ここでは、各弱学習器が二値決定変数であり、目的関数は正則化項を通じて精度とスパース性のバランスを取ります。`QuantumEnhancedEnsembleClassifier`はIBMハードウェア上でQAOAを使用してこの問題を解きますが、シミュレータベースの探索も可能です。`optimizer_options`はハイブリッドループを制御します：`simulator=False`は回路を選択したQPUにルーティングし、`num_solutions`は探索幅を増加させ、`classical_optimizer_options`（内部の古典オプティマイザ用）は収束を制御します。約60回の反復が品質と実行時間のバランスとして適切です。ランタイムオプション（適度な回路深度（`reps`）や標準的なトランスパイレーション努力など）は、デバイス間で堅牢な性能を確保するのに役立ちます。以下の構成はハードウェア実行に使用する「最良結果」プロファイルです。`simulator=True`に切り替えることで、QPU時間を消費せずにワークフローをドライランするための純粋なシミュレーションバリアントを作成することもできます。

In [12]:
# Problem scale and regularization
NUM_LEARNERS = 30
REGULARIZATION = 7

In [13]:
# ----- Quantum-enhanced ensemble on IBM hardware -----
print("\n-- Submitting quantum-enhanced ensemble job --")
job_2 = singularity.run(
    action="create_fit_predict",
    name="grid_stability_qeec",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=NUM_LEARNERS,
    regularization=REGULARIZATION,
    optimizer_options=optimizer_options,  # from Step 2
    backend_name=backend,  # least-busy compatible backend
    instance=IBM_INSTANCE_QUANTUM,
    random_state=RANDOM_STATE,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_test=X_test,
    fit_params={"validation_data": (X_val, y_val)},
    options={"save": False},
)
result_2 = job_2.result()
print("Action status:", result_2.get("status"))
print("Action message:", result_2.get("message"))
print("QPU Time:", result_2.get("metadata"))
qeec_pred_job_2 = np.array(result_2["data"]["predictions"])
_ = evaluate_predictions(qeec_pred_job_2, y_test)


-- Submitting quantum-enhanced ensemble job --
Action status: ok
Action message: Classifier created, fitted, and predicted.
QPU Time: {'resource_usage': {'RUNNING: MAPPING': {'CPU_TIME': 680.2116754055023}, 'RUNNING: WAITING_QPU': {'CPU_TIME': 80.80395102500916}, 'RUNNING: POST_PROCESSING': {'CPU_TIME': 154.4466371536255}, 'RUNNING: EXECUTING_QPU': {'QPU_TIME': 1095.822762966156}}}
Accuracy: 0.8946666666666667
Precision: 1.0
Recall: 0.8946666666666667
F1: 0.944405348346235


In [14]:
status_2 = job_2.status()
print("\nQuantum job status:", status_2)


Quantum job status: DONE


### Regularization

In this configuration, we increase to `num_learners = 60` and introduce adaptive regularization to manage sparsity more intuitively.

- With `regularization = "auto"`, the optimizer automatically finds a suitable regularization strength that selects approximately `regularization_ratio * num_learners` weak learners for the final ensemble, rather than fixing the penalty manually. This provides a more convenient interface for managing the balance between sparsity and ensemble size.
- `regularization_type = "alpha"` defines how the penalty is applied. Unlike `onsite`, which is unbounded `[0, ∞]`, `alpha` is bounded between `[0, 1]`, making it easier to tune and interpret. The parameter controls the trade-off between individual and pairwise penalties, offering a smoother configuration range.
- `regularization_desired_ratio ≈ 0.82` specifies the target proportion of learners to keep active after regularization — here, around 82% of learners are retained, trimming the weakest 18% automatically.

While adaptive regularization simplifies configuration and helps maintain a balanced ensemble, it does not necessarily guarantee better or more stable performance. The actual quality depends on selecting an appropriate regularization parameter, and fine-tuning it through cross-validation can be computationally expensive. The main advantage lies in improved usability and interpretability rather than direct accuracy gains.

In [15]:
# Problem scale and regularization
NUM_LEARNERS = 60
REGULARIZATION = "auto"
REGULARIZATION_TYPE = "alpha"
REGULARIZATION_RATIO = 0.82

In [16]:
# ----- Quantum-enhanced ensemble on IBM hardware -----
print("\n-- Submitting quantum-enhanced ensemble job --")
job_3 = singularity.run(
    action="create_fit_predict",
    name="grid_stability_qeec",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=NUM_LEARNERS,
    regularization=REGULARIZATION,
    regularization_type=REGULARIZATION_TYPE,
    regularization_desired_ratio=REGULARIZATION_RATIO,
    optimizer_options=optimizer_options,  # from Step 2
    backend_name=backend,  # least-busy compatible backend
    instance=IBM_INSTANCE_QUANTUM,
    random_state=RANDOM_STATE,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_test=X_test,
    fit_params={"validation_data": (X_val, y_val)},
    options={"save": False},
)
result_3 = job_3.result()
print("Action status:", result_3.get("status"))
print("Action message:", result_3.get("message"))
print("Metadata:", result_3.get("metadata"))
qeec_pred_job_3 = np.array(result_3["data"]["predictions"])
_ = evaluate_predictions(qeec_pred_job_3, y_test)


-- Submitting quantum-enhanced ensemble job --
Action status: ok
Action message: Classifier created, fitted, and predicted.
Metadata: {'resource_usage': {'RUNNING: MAPPING': {'CPU_TIME': 1387.7451872825623}, 'RUNNING: WAITING_QPU': {'CPU_TIME': 95.41597843170166}, 'RUNNING: POST_PROCESSING': {'CPU_TIME': 171.78878355026245}, 'RUNNING: EXECUTING_QPU': {'QPU_TIME': 1146.5584812164307}}}
Accuracy: 0.908
Precision: 1.0
Recall: 0.908
F1: 0.9517819706498952


In [17]:
status_3 = job_3.status()
print("\nQuantum job status:", status_3)


Quantum job status: DONE


## Step 4: Post-process and return result in desired classical format

We now post-process outputs from both the classical and quantum runs, converting them into a consistent format for downstream evaluation. This step compares predictive quality using standard metrics - accuracy, precision, recall, and F1 - and analyzes how ensemble width (`num_learners`) and sparsity control (`regularization`) influence both performance and computational behavior.

The classical AdaBoost baseline provides a compact and stable reference for small-scale learning. It performs well with limited ensembles and negligible compute overhead, reflecting the strength of traditional boosting when the hypothesis space is still tractable. The quantum configurations (`qeec_pred_job_1`, `qeec_pred_job_2`, and `qeec_pred_job_3`) extend this baseline by embedding the ensemble-selection process within a variational quantum optimization loop. This allows the system to explore exponentially large subsets of learners simultaneously in superposition, addressing the combinatorial nature of ensemble selection more efficiently as scale increases.

Results show that increasing `num_learners` from 10 to 30 improves recall and F1, confirming that a wider ensemble captures richer interactions among weak learners. The gain is sublinear on current hardware - each additional learner yields smaller accuracy increments - but the underlying scaling behavior remains favorable because the quantum optimizer can search broader configuration spaces without the exponential blow-up typical of classical subset selection. Regularization introduces additional nuance: a fixed λ=7 enforces consistent sparsity and stabilizes convergence, whereas adaptive α-regularization automatically tunes sparsity based on correlations between learners. This dynamic pruning often achieves slightly higher F1 for the same qubit width, balancing model complexity and generalization.

When compared directly with the AdaBoost baseline, the smallest quantum configuration (L=10) reproduces similar accuracy, validating the hybrid pipeline’s correctness. At larger widths, quantum variants - especially with auto-regularization - begin to surpass the classical baseline modestly, showing improved recall and F1 without linear growth in computational cost. These improvements do not indicate immediate "quantum advantage" but rather **scaling efficiency**: the quantum optimizer maintains tractable performance as the ensemble expands, where a classical approach would face exponential growth in subset-selection complexity.

In practice:
- Use the **classical baseline** for quick validation and benchmarking on small datasets.
- Apply **quantum ensembles** when model width or feature complexity grows—QAOA-based search scales more gracefully in those regimes.
- Employ **adaptive α-regularization** to maintain sparsity and generalization without increasing circuit width.
- Monitor QPU time and depth to balance quality gains against near-term hardware constraints.

Together, these experiments show that quantum-optimized ensembles complement classical methods: they reproduce baseline accuracy at small scales while offering a path to efficient scaling on larger, combinatorial learning problems. As hardware improves, these scaling advantages are expected to compound, extending the feasible size and depth of ensemble-based models beyond what is classically practical.

### Evaluate metrics for each configuration

We now evaluate all configurations - the classical AdaBoost baseline and the three quantum ensembles - using the `evaluate_predictions` helper to compute accuracy, precision, recall, and F1 on the same test set. This comparison clarifies how quantum optimization scales relative to the classical approach: at small widths, both perform similarly; as ensembles grow, the quantum method can explore larger hypothesis spaces more efficiently. The resulting table captures these trends in a consistent, quantitative form.

In [18]:
results = []

# Classical baseline
acc_b, prec_b, rec_b, f1_b = evaluate_predictions(baseline_pred, y_test)
results.append(
    {
        "Config": "AdaBoost (Classical)",
        "Accuracy": acc_b,
        "Precision": prec_b,
        "Recall": rec_b,
        "F1": f1_b,
    }
)

# Quantum runs
for label, preds in [
    ("QEEC L=10, reg=7", qeec_pred_job_1),
    ("QEEC L=30, reg=7", qeec_pred_job_2),
    (f"QEEC L=60, reg=auto (α={REGULARIZATION_RATIO})", qeec_pred_job_3),
]:
    acc, prec, rec, f1 = evaluate_predictions(preds, y_test)
    results.append(
        {
            "Config": label,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1": f1,
        }
    )


df_results = pd.DataFrame(results)
df_results

Accuracy: 0.7893333333333333
Precision: 1.0
Recall: 0.7893333333333333
F1: 0.8822652757078987
Accuracy: 0.868
Precision: 1.0
Recall: 0.868
F1: 0.9293361884368309
Accuracy: 0.8946666666666667
Precision: 1.0
Recall: 0.8946666666666667
F1: 0.944405348346235
Accuracy: 0.908
Precision: 1.0
Recall: 0.908
F1: 0.9517819706498952


,Config,Accuracy,Precision,Recall,F1
0,AdaBoost (Classical),0.789333,1.0,0.789333,0.882265
1,"QEEC L=10, reg=7",0.868000,1.0,0.868000,0.929336
2,"QEEC L=30, reg=7",0.894667,1.0,0.894667,0.944405
3,"QEEC L=60, reg=auto (α=0.82)",0.908000,1.0,0.908000,0.951782


### 学習器数の増加
ここでは `num_learners` を10から30に増加させ、`regularization = 7` は維持します。

- 学習器を増やすと仮説空間が拡大し、モデルがより微妙なパターンを捉えられるようになり、F1がわずかに向上する可能性があります。
- ほとんどの場合、10個と30個の学習器間の実行時間の差は大きくなく、回路幅の増加が実行コストを大幅には増加させないことを示しています。
- 品質の向上は依然として*収穫逓減曲線*に従います。アンサンブルが成長するにつれて初期の改善が現れますが、追加の学習器が提供する新しい情報が減少するにつれてプラトーに達します。

この実験は品質と効率のトレードオフを明らかにします。アンサンブル幅を増加させると、バックエンドとトランスパイレーション条件に応じて、大きな実行時間のペナルティなしにわずかな精度向上が得られる場合があります。

In [19]:
x = np.arange(len(df_results))
width = 0.35
plt.figure(figsize=(7.6, 4.6))
plt.bar(x - width / 2, df_results["Accuracy"], width=width, label="Accuracy")
plt.bar(x + width / 2, df_results["F1"], width=width, label="F1")
plt.xticks(x, df_results["Config"], rotation=10)
plt.ylabel("Score")
plt.title("Classical vs Quantum ensemble performance")
plt.legend()
plt.ylim(0, 1.0)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/sml-classification/extracted-outputs/0f15c5fb-2450-4671-9bc2-471043414df2-0.avif" alt="Output of the previous code cell" />

### Interpretation

The plot confirms the expected scaling pattern. The classical AdaBoost performs strongly for smaller ensembles but becomes increasingly costly to scale as the number of weak learners grows, because its subset-selection problem expands combinatorially. The quantum-enhanced models replicate classical accuracy at low widths and begin to surpass it as ensemble size increases, especially under adaptive α-regularization. This reflects the quantum optimizer’s ability to sample and evaluate many candidate subsets in parallel through superposition, maintaining tractable search even at higher widths. While current hardware overhead offsets some of the theoretical gains, the trend illustrates the scaling efficiency advantage of the quantum formulation. In practical terms, the classical method remains preferable for lightweight benchmarks, while quantum-enhanced ensembles become advantageous as model dimensionality and ensemble size expand, offering better trade-offs between accuracy, generalization, and computational growth.

## Appendix: Scaling benefits and enhancements

The scalability advantage of the `QuantumEnhancedEnsembleClassifier` arises from how the ensemble-selection process maps to quantum optimization.
Classical ensemble learning methods, such as AdaBoost or random forests, become computationally expensive as the number of weak learners increases because selecting the optimal subset is a combinatorial problem that scales exponentially.

In contrast, the quantum formulation — implemented here via the Quantum Approximate Optimization Algorithm (QAOA) — can explore these exponentially large search spaces more efficiently by evaluating multiple configurations in superposition.
As a result, the training time does not grow significantly with the number of learners, allowing the model to remain efficient even as ensemble width increases.

While current hardware introduces some noise and depth limitations, this workflow demonstrates a near-term hybrid approach where classical and quantum components cooperate: the quantum optimizer provides a better initialization landscape for the classical loop, improving convergence and final model quality.
As quantum processors evolve, these scalability benefits are expected to extend to larger datasets, broader ensembles, and deeper circuit depths.

## References

1. [Introduction to Qiskit Functions](/docs/guides/functions)
2. [Multiverse Computing Singularity Machine Learning](/docs/guides/multiverse-computing-singularity)

## Tutorial survey

Please take a minute to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3BLFkNVEuh0QBWm)